In [ ]:
# Creating a new ABM using a network structure

In [12]:
# loading packages
import random
import numpy as np
import networkx as nx

import mesa
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer
from mesa.datacollection import DataCollector

# CPS workflow

## World 


## Agents
Universal parameters:
problem_solving
communication

role_correct
role_distribution
role_negotiation lvl3
role_monitoring


Level 1

Level 2

Level 3
role_distributed=1

Level 4
Role_effect=1





## Teams are formed

## Problems
A team is posed with a problem with Task_complexity (1:3) and Size (1000)


Then we distribute role: 1 to len(team) (Do you fulfill your role or not?)
- each is assigned a "correct" role
- Then roles are more probable to become correct based on lvl2-4

- If Lvl 3/4 then roles are distributed correctly/evenly at first
- Self.Role=X

First we try to generate common-ground:
- Based on agents Communication parameter
- Common_ground = 0 or 1

Then we simulate role_alignment:
- There some 0.8 prob that role stays correct
- Theres some 0.2 prob that role might shift to incorrect

Then Solutions are generated:
- generate some 50 to 150 solution (based on problem_solving)
- If lvl doesnt match task lvl then solution cannot be given

If some role > 1:
- Then conflict appears

If common_ground=0
- Lowest solution is chosen

If all roles are evenly distributed and common ground = 1:
    Then choose the 

Adjustement of error/Meta negotiations/Self and team monitoring:
- Resolve_conflict
    - 50% prob of conflict resolvement
    - 90% if lvl 3 is present
- Reach_common_ground
    - 


In [100]:
class TeamAgent(Agent):
    """ An agent with fixed initial wealth."""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.cps_level = random.randint(1, 4)
        self.problem_solving = random.uniform(0, 1)
        self.communication = random.randint(0, 1)
        self.team_members = []

    def team(self):
         # Generate teams
        
        
        
        for node in self.model.network.G.nodes:
            team_size = random.randint(2, 5)
            

            neighbor_nodes = self.model.network.get_neighbors(node, include_center=False)

            neighbors = self.model.network.get_cell_list_contents(neighbor_nodes)
            random.shuffle(neighbors)

            for neighbor in neighbors:
                connection_strength = (self.communication + neighbor.communication) / 2
                if len(self.team_members) < team_size and connection_strength >= random.random():
                    self.team_members.append(neighbor)

            # Perform team actions
            
             ##   print("Team formed:", [member.unique_id for member in team_members])
        

    def problem(self, team_members):
        # nothing yet
        pass

    def get_team(self):
        return self.team_members
        
    def step(self):
        self.team_members = []
        self.team()
        
        

        
       

In [101]:
# Creating the model
class TeamModel(Model):
    def __init__(self, num_agents):
        self.num_agents = num_agents
        self.schedule = RandomActivation(self)
        
        self.teams = {}
        
        graph = nx.gnm_random_graph(self.num_agents, self.num_agents * 2)
        self.network = mesa.space.NetworkGrid(
            graph)
        
        
        
        for i in range(self.num_agents):
            agent = TeamAgent(i, self)
            self.schedule.add(agent)
            self.network.place_agent(agent, i)

    # A function which checks
    def collect_teams(self):
        teams = {}
        for agent in self.schedule.agents:
            teams[agent.unique_id] = agent.get_team()

        # Print teams for each agent
        for agent_id, team in teams.items():
            print("Agent", agent_id, "Team formed:", [member.unique_id for member in team])

        # Evaluation
        all_distributed = all(len(team) >= 2 for team in teams.values())
        no_duplicate_agents = all(
            agent_id not in [member.unique_id for team in teams.values() for member in team] for agent_id in teams.keys()
        )
        if all_distributed and no_duplicate_agents:
            print("Evaluation: All agents are distributed in teams of minimum two people, and no team holds the same agent ID twice.")
        else:
            print("Evaluation: Agents are not properly distributed in teams.")

    def step(self):
        self.schedule.step()
        self.collect_teams()


    

In [102]:

model = TeamModel(num_agents=20)

for i in range(5):
    model.step()

Agent 0 Team formed: [0, 12, 0, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Agent 1 Team formed: [1, 7, 8, 1, 11, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Agent 2 Team formed: [2, 12, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
Agent 3 Team formed: [3, 8, 12, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
Agent 4 Team formed: [4, 4, 8, 4, 17, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
Agent 5 Team formed: [5, 12, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
Agent 6 Team formed: [6, 12, 6, 17, 4, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
Agent 7 Team formed: [7, 4, 12, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]
Agent 8 Team formed: [8, 4, 12, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]
Agent 9 Team formed: [9, 8, 4, 12, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
Agent 10 Team formed: [10, 12, 8, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,